In [ ]:
from harmony.harmony import HarmonyJob, Harmony
from getpass import getpass
from cmr import CollectionQuery

In [ ]:
cmr = CollectionQuery()
q = cmr.keyword("SMAP")
q.get(3)

In [ ]:
# collection_id = 'C1227495594-LANCEMODIS'
collection_id = 'C1940468263-POCLOUD'
#-106.029053,40.304665,-105.227051,40.713956

#     'lat':'(-2:6)',
#     'lon':'(-52:-43)',

harmony_params = {
    'collection_id': collection_id,
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'lat':'(40:41)',
    'lon':'(-107:-105)',
    'start': '2020-06-01T00:00:00Z',
    'stop':'2020-06-30T23:59:59Z',
    'format': 'application/x-zarr',
}

password = getpass('Password: ')

credentials = {
    'user': 'betolink',
    'password': password
}

client = Harmony(credentials)

In [ ]:
# collection info
client.collection(collection_id)

In [ ]:
# request URI
client.subset_uri(harmony_params)

In [ ]:
# subset Harmony
job = client.subset(harmony_params)

In [ ]:
# job status
job.status()

In [ ]:
# Job stac uri
job.stac_uri()

In [ ]:
outzarr = job.s3_file_uris()

In [ ]:
# Pystac catalog
cat = job.stac_catalog()

In [ ]:
# cat.describe(include_hrefs=True)
from pprint import pprint
items = cat.get_all_items()
for item in items:
#     pprint(item.__dict__)
    for k in item.assets:
        print(k)

In [ ]:
outzarr[1]

In [ ]:
job.s3_download_file(outzarr[1])

In [ ]:
import s3fs
import zarr

s3_keys = client.aws_keys


zarr_fs = s3fs.S3FileSystem(
    key=s3_keys['AccessKeyId'],
    secret=s3_keys['SecretAccessKey'],
    token=s3_keys['SessionToken'],
    client_kwargs={'region_name':'us-west-2'},
)

zarr_store = zarr_fs.get_mapper(root=outzarr[1], check=False)

zarr_dataset = zarr.open(zarr_store)

print(zarr_dataset.tree())

In [ ]:
# xarray

import xarray as xr
ds = xr.open_zarr(zarr_store)
print(ds)